In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import xlrd

In [2]:
#2021 import
daily_production_df = pd.read_csv('data.csv', index_col='Unnamed: 0')

#Chane Date column type to datetype (When in was inported, it was an obj)
daily_production_df.Date = pd.to_datetime(daily_production_df.Date) #changes Date to DateTime type from Object(string) type
#df.Date = pd.to_datetime(df.Date) # TO DO: CHECK DOCUMENTATION FOR THIS

#removing NaNs
daily_production_df['Remarks'] = daily_production_df['Remarks'].fillna('') #Used this method to only execute on Remarks column,
#so DateTime and float64 types wouldn't be converted to Object types


#Formating floats to show zero decimals
pd.options.display.float_format = '{:,.0f}'.format

#daily_production_df

In [3]:
#2021 dates
import datetime
end = datetime.date.today() #THIS FORMAT WORKS
#end = datetime.datetime(2021, 2, 1).date()
start = daily_production_df.iloc[0]['Date'].date() # Gets the most recent date from the dataframe

numdays = (end-start).days # Calculates the number of days between the above two dates

date_list = [end - timedelta(days=x) for x in range(numdays)]

In [4]:
import_2021_df = pd.DataFrame()

for i in range(numdays):
    try:
        path2021 = 'G:\CML Operations\WELL FILES\A     NON-OPERATED - SOLD\HAWKWOOD\Daily production starting 5-17-2018\\2021\DailyProductionReport-' + str(date_list[i]) + '.xlsx'
        import_2021 = pd.read_excel(path2021)
        import_2021['Remarks'] = import_2021['Remarks'].fillna('')
        import_2021.dropna(axis=0, how='any', inplace=True)
        

        
        #import_2021_df = pd.concat([import_2021_df, import_2021]).drop_duplicates()
        import_2021_df = pd.concat([import_2021_df,import_2021]).drop_duplicates()
    except:
        print('The production for ' + str(date_list[i]) + ' is not available.')
        continue

The production for 2021-02-23 is not available.
The production for 2021-02-22 is not available.
The production for 2021-02-21 is not available.
The production for 2021-02-20 is not available.
The production for 2021-02-19 is not available.
The production for 2021-02-18 is not available.
The production for 2021-02-17 is not available.
The production for 2021-02-16 is not available.
The production for 2021-02-15 is not available.
The production for 2021-02-14 is not available.
The production for 2021-02-13 is not available.
The production for 2021-02-12 is not available.
The production for 2021-02-11 is not available.


In [5]:
try:
    import_2021_df.Date = pd.to_datetime(import_2021_df.Date)
    import_2021_df['Date'] = pd.to_datetime(import_2021_df["Date"].dt.strftime('%Y-%m-%d'))
    import_2021_df.to_csv('plot.csv')
except AttributeError:
    print("Cannot create dataframe for " + str(date_list[i]) + " production")

#import_2021_df

Cannot create dataframe for 2021-02-11 production


In [ ]:
import_2021_df.to_csv('plot.csv') #SAVE DAILY NUMBERS TO MACRO THEM TO DESIGNATED TABS IN THE Daily Production by well.xlsx

In [6]:
# concatenating 2021 import to daily production df (2018, 2019, &2020)
daily_production_df = pd.concat([daily_production_df, import_2021_df]).drop_duplicates()

#sorting data by name_site ascending, keeping most recent date at ['Date'][0] so it can be refernced in the following yearts date list
daily_production_df = daily_production_df.sort_values(['Date', 'Site_Name'], ascending = [False , True]) 

#daily_production_df

In [ ]:
#saving 2021 production to csv
daily_production_df.to_csv('data.csv')

daily_production_df.to_csv('T:\Geotech-Documents\Hawkwood_Production_Autonomation\data.csv')

In [7]:
import ipywidgets as widgets
from IPython.display import display, HTML
from ipywidgets import interact, Dropdown, interact_manual



import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
#%matplotlib inline
#%matplotlib widget
#%matplotlib nbagg

In [8]:
from mpldatacursor import datacursor
import mplcursors

In [9]:
# Create wellname and date widgets to sort dataframe
well_list = daily_production_df['Site_Name'].unique().tolist()

In [11]:
import matplotlib.pyplot as plt
%matplotlib widget

#wellname = 'CROC 1H'
wellnameW = widgets.Dropdown(
    options=well_list,
    description='Well Site:',
)



@interact_manual(wellname = wellnameW)



def main(wellname):
    df_query = daily_production_df[(daily_production_df['Site_Name'] == wellname) &
            (daily_production_df['Oil_(BBL)'] > 0) & 
            (daily_production_df['Gas_(MCF)'] > 0) & 
            (daily_production_df['Water_(BBL)'] > 0)]
    
    # Get cum for entire well life regardless of days !!selected. That way cum oil is always correct
    df_cum = daily_production_df[daily_production_df['Site_Name'] == wellname]
    cumoil = int(df_cum['Oil_(BBL)'].sum())
    cumgas = int(df_cum['Gas_(MCF)'].sum())
    print(f'{wellname} has produced {cumoil} BBLS of oil')
    print(f'{wellname} has produced {cumgas} MCF of gas')
    
    # Params
    plt.rcParams['figure.figsize'] = [25, 20]
    well_outputs = ['Oil_(BBL)', 'Gas_(MCF)', 'Water_(BBL)']
    output_colors = ['g', 'r','b']
    
    fig, axs= plt.subplots(3, constrained_layout=True)
    # Make 3 subplots for oil, water, and gas
    for i in range(3):
        fontdict={'fontsize': 20}
        axs[i].plot(df_query['Date'].tolist(), df_query[well_outputs[i]], color = output_colors[i])
        axs[i].set_title(f'{wellname} {well_outputs[i]} Production', fontdict=fontdict, pad=15.0)
        axs[i].grid()
        axs[i].set_yscale('log')      

    mplcursors.cursor()
    #fig.tight_layout() 
    plt.show()
  
 

interactive(children=(Dropdown(description='Well Site:', options=('CROC 1H', 'GALAH 3H', 'GOANNA 1H', 'GREAT W…